## first level analysis pipeline using nipype

In [2]:
from os.path import join as opj
import json
from nipype.interfaces.spm import Level1Design, EstimateModel, EstimateContrast
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype import Workflow, Node

### set up

In [3]:
experiment_dir = '/output'
output_dir = 'first_level'
working_dir = 'workingdir'

TR = 2. # TR of functional images

### specify nodes

In [4]:
# SpecifyModel - Generates SPM-specific Model
modelspec = Node(SpecifySPMModel(concatenate_runs=False,
                                 input_units='secs',
                                 output_units='secs',
                                 time_repetition=TR,
                                 high_pass_filter_cutoff=128),
                 name="modelspec")

# Level1Design - Generates an SPM design matrix
level1design = Node(Level1Design(bases={'hrf': {'derivs': [1, 0]}},
                                 timing_units='secs',
                                 interscan_interval=TR,
                                 model_serial_correlations='FAST'),
                    name="level1design")

# EstimateModel - estimate the parameters of the model
level1estimate = Node(EstimateModel(estimation_method={'Classical': 1}),
                      name="level1estimate")

# EstimateContrast - estimates contrasts
level1conest = Node(EstimateContrast(), name="level1conest")

### specify GLM contrasts

sanity check of high vs low pain

In [5]:
# condition names
condition_names = ['High','Low']

# contrasts
cont01 = ['average', 'T', condition_names, [.5, .5]]
cont02 = ['High',    'T', condition_names, [1, 0]]
cont03 = ['Low',     'T', condition_names, [0, 1]]
cont04 = ['High>Low','T', condition_names, [.5,-.5]]
cont05 = ['Low>High','T', condition_names, [-.5,.5]]

contrast_list = [cont01, cont02, cont03, cont04, cont05]

### specify GLM model

In [7]:
import pandas as pd
trialinfo = pd.read_csv('../data/fmri_behavioural.csv')
trialinfo.head(10)

,subject,session,trial,seq,rt,gen_p1g2,gen_p2g1,prob_pxl,prob_obs,obs_p1g2,obs_p2g1,obs_p1g1,obs_p2g2,obs_p1,obs_p2,obs_p,runtime
0,6,1,1,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.60
1,6,1,2,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.10
2,6,1,3,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.60
3,6,1,4,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.05
4,6,1,5,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.60
5,6,1,6,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.20
6,6,1,7,1,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.80
7,6,1,8,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.25
8,6,1,9,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.80
9,6,1,10,2,NaN,0.2,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.40


In [24]:
# construct df
df_sj6 = trialinfo[(trialinfo['subject']==6) & (trialinfo['session']==1)]
sj_info = pd.DataFrame()
sj_info['onset'] = df_sj6['runtime']
sj_info['duration'] = 0.
sj_info['weight'] = 1.
trial_type = df_sj6['seq'].replace({1:'Low', 2:'High'})
sj_info['trial_type'] = trial_type
sj_info

,onset,duration,weight,trial_type
0,1.60,0.0,1.0,High
1,3.10,0.0,1.0,High
2,4.60,0.0,1.0,High
3,6.05,0.0,1.0,High
4,7.60,0.0,1.0,High
...,...,...,...,...
255,469.05,0.0,1.0,High
256,470.45,0.0,1.0,Low
257,471.85,0.0,1.0,High
258,473.40,0.0,1.0,Low


In [25]:
for group in sj_info.groupby('trial_type'):
    print(group)
    print("")

('High',       onset  duration  weight trial_type
0      1.60       0.0     1.0       High
1      3.10       0.0     1.0       High
2      4.60       0.0     1.0       High
3      6.05       0.0     1.0       High
4      7.60       0.0     1.0       High
..      ...       ...     ...        ...
250  461.45       0.0     1.0       High
251  463.00       0.0     1.0       High
254  467.45       0.0     1.0       High
255  469.05       0.0     1.0       High
257  471.85       0.0     1.0       High

[131 rows x 4 columns])

('Low',       onset  duration  weight trial_type
6     10.80       0.0     1.0        Low
10    16.90       0.0     1.0        Low
16    25.90       0.0     1.0        Low
22    41.70       0.0     1.0        Low
24    44.80       0.0     1.0        Low
..      ...       ...     ...        ...
252  464.45       0.0     1.0        Low
253  465.90       0.0     1.0        Low
256  470.45       0.0     1.0        Low
258  473.40       0.0     1.0        Low
259  474.95   

### specify helper function to read subject info

In [26]:
def construct_sj(trialinfo, subject_id):
    # construct df
    df_sj = trialinfo[(trialinfo['subject']==subject_id) & (trialinfo['session']==1)]
    sj_info = pd.DataFrame()
    sj_info['onset'] = df_sj['runtime']
    sj_info['duration'] = 0.
    sj_info['weight'] = 1.
    trial_type = df_sj['seq'].replace({1:'Low', 2:'High'})
    sj_info['trial_type'] = trial_type
    return sj_info

In [27]:
def subjectinfo(subject_id):

    import pandas as pd
    from nipype.interfaces.base import Bunch

    alltrialinfo = pd.read_csv('../data/fmri_behavioural.csv')
    alltrialinfo.head()
    trialinfo = construct_sj(trialinfo, subject_id)
    conditions = []
    onsets = []
    durations = []

    for group in trialinfo.groupby('trial_type'):
        conditions.append(group[0])
        onsets.append(list(group[1].onset - 10)) # subtracting 10s due to removing of 4 dummy scans
        durations.append(group[1].duration.tolist())

    subject_info = [Bunch(conditions=conditions,
                          onsets=onsets,
                          durations=durations,
                          #amplitudes=None,
                          #tmod=None,
                          #pmod=None,
                          #regressor_names=None,
                          #regressors=None
                         )]

    return subject_info  # this output will later be returned to infosource

# Get Subject Info - get subject specific condition information
getsubjectinfo = Node(Function(input_names=['subject_id'],
                               output_names=['subject_info'],
                               function=subjectinfo),
                      name='getsubjectinfo')

### specify input and output

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id',
#                                             'fwhm_id',
                                            'contrasts'],
                                    contrasts=contrast_list),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]#,
#                         ('fwhm_id', fwhm)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
# templates = {'func': opj(output_dir, 'preproc', 'sub-{subject_id}', 'task-{task_id}',
#                          'fwhm-{fwhm_id}_ssub-{subject_id}_ses-test_task-{task_id}_bold.nii'),
#              'mc_param': opj(output_dir, 'preproc', 'sub-{subject_id}', 'task-{task_id}',
#                              'sub-{subject_id}_ses-test_task-{task_id}_bold.par'),
#              'outliers': opj(output_dir, 'preproc', 'sub-{subject_id}', 'task-{task_id}',
#                              'art.sub-{subject_id}_ses-test_task-{task_id}_bold_outliers.txt')}
templates = {'func': opj(output_dir, 'preproc', 'sub-{subject_id}',
                         'sub-{subject_id}_ses-test_task-{task_id}_bold.nii'),
             'mc_param': opj(output_dir, 'preproc', 'sub-{subject_id}', 'task-{task_id}',
                             'sub-{subject_id}_ses-test_task-{task_id}_bold.par')}
selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir,
                               sort_filelist=True),
                   name="selectfiles")
selectfiles.inputs.task_id = 'fingerfootlips'

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', 'sub-')]
subjFolders = [('_fwhm_id_%ssub-%s' % (f, sub), 'sub-%s/fwhm-%s' % (sub, f))
               for f in fwhm
               for sub in subject_list]
substitutions.extend(subjFolders)
datasink.inputs.substitutions = substitutions

### specify workflow

In [ ]:
# Initiation of the 1st-level analysis workflow
l1analysis = Workflow(name='l1analysis')
l1analysis.base_dir = opj(experiment_dir, working_dir)

# Connect up the 1st-level analysis components
l1analysis.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                               ('fwhm_id', 'fwhm_id')]),
                    (infosource, getsubjectinfo, [('subject_id',
                                                   'subject_id')]),
                    (getsubjectinfo, modelspec, [('subject_info',
                                                  'subject_info')]),
                    (infosource, level1conest, [('contrasts', 'contrasts')]),
                    (selectfiles, modelspec, [('func', 'functional_runs')]),
                    (selectfiles, modelspec, [('mc_param', 'realignment_parameters'),
                                              ('outliers', 'outlier_files')]),
                    (modelspec, level1design, [('session_info',
                                                'session_info')]),
                    (level1design, level1estimate, [('spm_mat_file',
                                                     'spm_mat_file')]),
                    (level1estimate, level1conest, [('spm_mat_file',
                                                     'spm_mat_file'),
                                                    ('beta_images',
                                                     'beta_images'),
                                                    ('residual_image',
                                                     'residual_image')]),
                    (level1conest, datasink, [('spm_mat_file', '1stLevel.@spm_mat'),
                                              ('spmT_images', '1stLevel.@T'),
                                              ('con_images', '1stLevel.@con'),
                                              ('spmF_images', '1stLevel.@F'),
                                              ('ess_images', '1stLevel.@ess'),
                                              ]),
                    ])

### visualise workflow

In [ ]:
# Create 1st-level analysis output graph
l1analysis.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename=opj(l1analysis.base_dir, 'l1analysis', 'graph.png'))

### run workflow

In [ ]:
l1analysis.run('MultiProc', plugin_args={'n_procs': 8})